# Taller 4

<h4>1. Implemente un método que retorne la subsecuencia común más larga (SCL) apartir de la matriz C construida por el algoritmo de PD que calcula la longitud de la SCL.</h4>

<h4>2. Para el problema del corte de la barilla (Notebook): https://github.com/ejake/algorithm_analysis/blob/master/Rod-cutting.ipynb</h4>

<h6>Implemente un método para determinar los cortes que se deben realizar a partir de la solución de PD que encuentra el valor de la solución óptima.</h6>

<h6>Realice una comparación experimental sistemática de los algoritmos para diferentes valores de p y de n.</h6>

<h4>3. Implemente, describa y analice un algoritmo de PD para calcular la distancia de edición entre dos cadenas. (Cormen, problem 15-5)</h4>

<h4>4. En el rio atrato hay n embarcaderos, en cada uno de ellos se puede alquilar un bote para realizar un viaje rio abajo a otro embarcadero. Existe una tabla de tarifas que indica el costo del viaje del embarcadero i al j, para i, j = 1,2,...,n  (i menor j), puede pasar que un viaje de i a j sea más caro que una serie de viajes sucesivos intermedios, no hay un costo asociado a cambiar de bote en algún embarcadero. El problema consiste en diseñar un algoritmo de PD que determine el costo mínimo para cada par de puntos i,j.</h4>